In [1]:
#import all the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import preprocessing
%pylab inline

# To ignore some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

Populating the interactive namespace from numpy and matplotlib


# Load datasets

In [2]:
# Load ACS and garden data
ACS = pd.read_csv("../data/ACS_Merge_Norm_Final.csv",index_col=0)
# Load crime and garden data
crime = pd.read_csv("../data/Crime_per_NTA_capita.csv",index_col=0)

In [3]:
ACS.head()

,GeoType,NTA,GeoID,Borough,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
0,NTA2010,Bath Beach,BK27,Brooklyn,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
1,NTA2010,Bay Ridge,BK31,Brooklyn,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
2,NTA2010,Bedford,BK75,Brooklyn,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
3,NTA2010,Bensonhurst East,BK29,Brooklyn,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
4,NTA2010,Bensonhurst West,BK28,Brooklyn,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [4]:
ACS = ACS.set_index('NTA')
ACS.head()

,GeoType,GeoID,Borough,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,NTA2010,BK27,Brooklyn,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
Bay Ridge,NTA2010,BK31,Brooklyn,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
Bedford,NTA2010,BK75,Brooklyn,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
Bensonhurst East,NTA2010,BK29,Brooklyn,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
Bensonhurst West,NTA2010,BK28,Brooklyn,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [5]:
ACS = ACS.drop(columns=['GeoID','GeoType','Borough'])
ACS.head()

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,0.436933,0.012182,0.000332,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
Bay Ridge,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,0.634329,0.019094,0.002287,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
Bedford,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,0.350607,0.405826,0.000876,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
Bensonhurst East,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,0.465153,0.008262,0.000000,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
Bensonhurst West,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,0.426773,0.008740,0.001211,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [6]:
crime.head()

,FELONY,MISDEMEANOR,VIOLATION,TOTAL_CRIME,Population,GardenCounts,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
ntaname,,,,,,,,,,
Allerton-Pelham Gardens,752.0,1925.0,444.0,3121.0,32450.0,1.0,0.023174,0.059322,0.013683,0.096179
Annadale-Huguenot-Prince's Bay-Eltingville,431.0,1215.0,464.0,2110.0,28698.0,0.0,0.015018,0.042337,0.016168,0.073524
Arden Heights,263.0,785.0,324.0,1372.0,26482.0,0.0,0.009931,0.029643,0.012235,0.051809
Astoria,4002.0,8082.0,2211.0,14295.0,77252.0,1.0,0.051804,0.104619,0.028621,0.185044
Auburndale,557.0,713.0,258.0,1528.0,20609.0,0.0,0.027027,0.034597,0.012519,0.074142


# Merge datasets

In [7]:
# merge crime and ACS data
data = ACS.merge(crime[['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']], how='left', left_index=True, right_index = True)
data.head()

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,0.436933,0.012182,0.000332,...,23822.0,0.663798,0.224372,0.111829,0.0,0.0,0.030033,0.065645,0.014444,0.110123
Bay Ridge,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,0.634329,0.019094,0.002287,...,58178.0,0.535735,0.271976,0.192289,0.0,0.0,0.033627,0.082051,0.023833,0.139510
Bedford,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,0.350607,0.405826,0.000876,...,44766.0,0.678193,0.204352,0.117455,23.0,1.0,0.082026,0.124671,0.032993,0.239690
Bensonhurst East,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,0.465153,0.008262,0.000000,...,46010.0,0.698153,0.204564,0.097283,1.0,1.0,0.029377,0.048003,0.014969,0.092349
Bensonhurst West,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,0.426773,0.008740,0.001211,...,65016.0,0.737618,0.188800,0.073582,2.0,1.0,0.027704,0.045981,0.013105,0.086790


# Deal with null rows/columns

In [8]:
# check for nulls
data[data.isnull().any(axis=1)]
# probably want to drop airport and park-cemetery-etc-Staten Island	 since they have population=0
# want to fillna for Hudson Yards-Chelsea-Flat Iron-Union Square
# not sure why [Rikers Island, park-cemetery-etc-Manhattan] have null values

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
NTA,,,,,,,,,,,,,,,,,,,,,
Hudson Yards-Chelsea-Flat Iron-Union Square,77205.0,0.520174,0.479826,0.103983,0.115472,0.140380,0.859620,0.631449,0.057367,0.000661,...,62879.0,0.255793,0.398273,0.345934,0.0,0.0,NaN,NaN,NaN,NaN
park-cemetery-etc-Manhattan,1765.0,0.877620,0.122380,0.003399,0.094051,0.202833,0.797167,0.218697,0.511615,0.007932,...,1677.0,0.938581,0.042338,0.019082,2.0,1.0,0.760340,1.542210,0.287252,2.589802
Airport,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
park-cemetery-etc-Staten Island,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
Rikers Island,7503.0,0.932160,0.067840,0.035586,0.004398,0.298547,0.701453,0.067973,0.599227,0.001866,...,5322.0,0.985720,0.006576,0.007704,0.0,0.0,0.411036,0.278822,0.002532,0.692390


In [9]:
data.shape

(195, 39)

In [10]:
data.drop(index=['Airport', 'park-cemetery-etc-Staten Island'], inplace=True)
data.shape

(193, 39)

In [11]:
# assign crime nan to zero
data[['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']] = data[['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']].fillna(value=0)
data[data.isnull().any(axis=1)]


,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
NTA,,,,,,,,,,,,,,,,,,,,,
park-cemetery-etc-Manhattan,1765.0,0.87762,0.12238,0.003399,0.094051,0.202833,0.797167,0.218697,0.511615,0.007932,...,1677.0,0.938581,0.042338,0.019082,2.0,1.0,0.760340,1.542210,0.287252,2.589802
Rikers Island,7503.0,0.93216,0.06784,0.035586,0.004398,0.298547,0.701453,0.067973,0.599227,0.001866,...,5322.0,0.985720,0.006576,0.007704,0.0,0.0,0.411036,0.278822,0.002532,0.692390


In [12]:
data[data.columns[data.isnull().any(axis=0)]][data.isnull().any(axis=1)]
# null values are for housing features
# will drop them

,HHINC_U10N,HHINC_10t14N,HHINC_15t24N,HHINC_25t34N,HHINC_35t49N,HHINC_50t74N,HHINC_75t99N,HHINC_100t149N,HHINC_150t199N,HHINC_200plN,CLF_UnemployedN
NTA,,,,,,,,,,,
park-cemetery-etc-Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.461774
Rikers Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data.shape

(193, 39)

In [14]:
data.dropna(axis='index', inplace=True)
data.shape

(191, 39)

# Drop population columns (not needed for modelling)

In [15]:
data.drop(columns=['Pop_1E', 'PopOver25'], inplace=True)

# Aggregate sparse columns

In [16]:
# import pandas_profiling

# pandas_profiling.ProfileReport(data)

In [17]:
data['RaceOther'] = data.OtherRace1N + data.OtherRace2N + data.NativeN + data.PacificN

In [18]:
#created income features by combining certain income variables:
data['Income_Low'] = data[['HHINC_U10N', 'HHINC_10t14N', 'HHINC_15t24N', 'HHINC_25t34N', 'HHINC_35t49N']].sum(axis=1)
data['Income_Mid'] = data[['HHINC_50t74N', 'HHINC_75t99N']].sum(axis=1)
data['Income_High'] = data[['HHINC_100t149N', 'HHINC_150t199N', 'HHINC_200plN']].sum(axis=1)


In [19]:
data = data[['FemEN', 'PopOv65N', 
             'HispN', 'WhiteN', 'BlackN', 'AsianN', 'RaceOther',
             'Income_Low', 'Income_Mid', 'Income_High',
             'CLF_UnemployedN','EduOtherN', 'EduBachelor', 'EduGradProf',
             'FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm',
             'GardenCounts','HasGarden']]

# Scaling crime

In [20]:
from sklearn import preprocessing
crime_cols = ['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']

for col in crime_cols:
    data[col+'_scaled'] = preprocessing.StandardScaler().fit_transform(data[[col]])
    data[col+'_MMscaled'] = preprocessing.MinMaxScaler().fit_transform(data[[col]])

In [21]:
data.head()

,FemEN,PopOv65N,HispN,WhiteN,BlackN,AsianN,RaceOther,Income_Low,Income_Mid,Income_High,...,GardenCounts,HasGarden,FELONY_norm_scaled,FELONY_norm_MMscaled,MISDEMEANOR_norm_scaled,MISDEMEANOR_norm_MMscaled,VIOLATION_norm_scaled,VIOLATION_norm_MMscaled,TOTAL_CRIME_norm_scaled,TOTAL_CRIME_norm_MMscaled
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,0.496849,0.147393,0.176733,0.436933,0.012182,0.361095,0.013057,0.433409,0.299790,0.266801,...,0.0,0.0,-0.197696,0.010496,-0.178816,0.012187,-0.245165,0.016769,-0.190612,0.012089
Bay Ridge,0.521963,0.156444,0.178608,0.634329,0.019094,0.145513,0.022456,0.409413,0.289008,0.301580,...,0.0,0.0,-0.184781,0.011752,-0.148587,0.015232,-0.117467,0.027670,-0.157676,0.015315
Bedford,0.515508,0.077219,0.194933,0.350607,0.405826,0.030404,0.018229,0.575924,0.229287,0.194789,...,23.0,1.0,-0.010826,0.028667,-0.070053,0.023144,0.007119,0.038305,-0.045398,0.026313
Bensonhurst East,0.513234,0.157266,0.123283,0.465153,0.008262,0.386793,0.016509,0.510524,0.290981,0.198495,...,1.0,1.0,-0.200053,0.010267,-0.211325,0.008911,-0.238025,0.017379,-0.210532,0.010138
Bensonhurst West,0.512788,0.142625,0.145975,0.426773,0.008740,0.405168,0.013345,0.492862,0.285700,0.221438,...,2.0,1.0,-0.206067,0.009682,-0.215050,0.008536,-0.263376,0.015215,-0.216762,0.009528


# MinMaxScale number of gardens

In [22]:
data['GardenCounts_MMscaled'] = preprocessing.MinMaxScaler().fit_transform(data[['GardenCounts']])

# Cleaning column names

In [23]:
data = data.rename(columns={'FemEN': 'Gender_Female', 
                             'PopOv65N': 'Age_Over65', 
                            'LaborForceN': 'LaborForce',
                             'HispN': 'Race_Hisp', 
                             'WhiteN': 'Race_White', 
                             'BlackN': 'Race_Black', 
                             'AsianN': 'Race_Asian',
                             'RaceOther': 'Race_Other',
#                              'HHINC_U10N': 'Income_under10', 
#                              'HHINC_10t14N': 'Income_10to14', 
#                              'HHINC_15t24N': 'Income_15to24', 
#                              'HHINC_25t34N': 'Income_25to34',
#                              'HHINC_35t49N': 'Income_35to49', 
#                              'HHINC_50t74N': 'Income_50to74', 
#                              'HHINC_75t99N': 'Income_75to99', 
#                              'HHINC_100t149N': 'Income_100to149',      
#                              'HHINC_150t199N': 'Income_150to199', 
#                              'HHINC_200plN': 'Income_200over', 
                             'CLF_UnemployedN': 'Unemployed', 
                             'EduOtherN': 'Edu_UnderBach',
                             'EduBachelor': 'Edu_Bachelor', 
                             'EduGradProf': 'Edu_GradProf', 
                             'FELONY_norm': 'Crime_Felonies_unscaled', 
                             'MISDEMEANOR_norm': 'Crime_Misdemeanor_unscaled',
                             'VIOLATION_norm': 'Crime_Violation_unscaled', 
                             'TOTAL_CRIME_norm': 'Crime_Total_unscaled', 
                             'FELONY_norm_scaled': 'Crime_Felony_scaled', 
                             'MISDEMEANOR_norm_scaled': 'Crime_Misdemeanor_scaled',
                             'VIOLATION_norm_scaled': 'Crime_Violation_scaled', 
                             'TOTAL_CRIME_norm_scaled': 'Crime_Total_scaled',
                             'FELONY_norm_MMscaled': 'Crime_Felony_MMscaled', 
                             'MISDEMEANOR_norm_MMscaled': 'Crime_Misdemeanor_MMscaled',
                             'VIOLATION_norm_MMscaled': 'Crime_Violation_MMscaled', 
                             'TOTAL_CRIME_norm_MMscaled': 'Crime_Total_MMscaled'})

In [24]:
# reorder columns
data = data[['HasGarden', 'GardenCounts_MMscaled', 
             'Gender_Female', 'Age_Over65', 'Unemployed', # general
             'Race_Hisp', 'Race_White', 'Race_Black', 'Race_Asian', 'Race_Other', # race
             'Income_Low', 'Income_Mid', 'Income_High',
#              'Income_under10', 'Income_10to14', # income
#              'Income_15to24', 'Income_25to34', 'Income_35to49', 
#              'Income_50to74', 'Income_75to99', 'Income_100to149', 
#              'Income_150to199', 'Income_200over',
             'Edu_UnderBach', 'Edu_Bachelor', 'Edu_GradProf', # education
             'Crime_Felonies_unscaled', 'Crime_Misdemeanor_unscaled', # crimes
             'Crime_Violation_unscaled', 'Crime_Total_unscaled', 
             'Crime_Felony_scaled', 'Crime_Misdemeanor_scaled', 
             'Crime_Violation_scaled', 'Crime_Total_scaled',
             'Crime_Felony_MMscaled', 'Crime_Misdemeanor_MMscaled', 
             'Crime_Violation_MMscaled', 'Crime_Total_MMscaled']]
data.head()

,HasGarden,GardenCounts_MMscaled,Gender_Female,Age_Over65,Unemployed,Race_Hisp,Race_White,Race_Black,Race_Asian,Race_Other,...,Crime_Violation_unscaled,Crime_Total_unscaled,Crime_Felony_scaled,Crime_Misdemeanor_scaled,Crime_Violation_scaled,Crime_Total_scaled,Crime_Felony_MMscaled,Crime_Misdemeanor_MMscaled,Crime_Violation_MMscaled,Crime_Total_MMscaled
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,0.0,0.000000,0.496849,0.147393,0.070566,0.176733,0.436933,0.012182,0.361095,0.013057,...,0.014444,0.110123,-0.197696,-0.178816,-0.245165,-0.190612,0.010496,0.012187,0.016769,0.012089
Bay Ridge,0.0,0.000000,0.521963,0.156444,0.074922,0.178608,0.634329,0.019094,0.145513,0.022456,...,0.023833,0.139510,-0.184781,-0.148587,-0.117467,-0.157676,0.011752,0.015232,0.027670,0.015315
Bedford,1.0,0.638889,0.515508,0.077219,0.125027,0.194933,0.350607,0.405826,0.030404,0.018229,...,0.032993,0.239690,-0.010826,-0.070053,0.007119,-0.045398,0.028667,0.023144,0.038305,0.026313
Bensonhurst East,1.0,0.027778,0.513234,0.157266,0.085202,0.123283,0.465153,0.008262,0.386793,0.016509,...,0.014969,0.092349,-0.200053,-0.211325,-0.238025,-0.210532,0.010267,0.008911,0.017379,0.010138
Bensonhurst West,1.0,0.055556,0.512788,0.142625,0.088342,0.145975,0.426773,0.008740,0.405168,0.013345,...,0.013105,0.086790,-0.206067,-0.215050,-0.263376,-0.216762,0.009682,0.008536,0.015215,0.009528


# MinMaxScale all datasets

In [25]:
df = pd.read_csv('../data/Data_crimeUnscaled.csv', index_col=0)
df.head()

,HasGarden,GardenCounts_MMscaled,Gender_Female,Age_Over65,Unemployed,Race_Hisp,Race_White,Race_Black,Race_Asian,Race_Other,Income_Low,Income_Mid,Income_High,Edu_Other,Edu_Bachelor,Edu_GradProf,Crime_Felonies_unscaled,Crime_Misdemeanor_unscaled,Crime_Violation_unscaled,Crime_Total_unscaled
NTA,,,,,,,,,,,,,,,,,,,,
Bath Beach,0.0,0.000000,0.496849,0.147393,0.070566,0.176733,0.436933,0.012182,0.361095,0.013057,0.433409,0.299790,0.266801,0.663798,0.224372,0.111829,0.030033,0.065645,0.014444,0.110123
Bay Ridge,0.0,0.000000,0.521963,0.156444,0.074922,0.178608,0.634329,0.019094,0.145513,0.022456,0.409413,0.289008,0.301580,0.535735,0.271976,0.192289,0.033627,0.082051,0.023833,0.139510
Bedford,1.0,0.638889,0.515508,0.077219,0.125027,0.194933,0.350607,0.405826,0.030404,0.018229,0.575924,0.229287,0.194789,0.678193,0.204352,0.117455,0.082026,0.124671,0.032993,0.239690
Bensonhurst East,1.0,0.027778,0.513234,0.157266,0.085202,0.123283,0.465153,0.008262,0.386793,0.016509,0.510524,0.290981,0.198495,0.698153,0.204564,0.097283,0.029377,0.048003,0.014969,0.092349
Bensonhurst West,1.0,0.055556,0.512788,0.142625,0.088342,0.145975,0.426773,0.008740,0.405168,0.013345,0.492862,0.285700,0.221438,0.737618,0.188800,0.073582,0.027704,0.045981,0.013105,0.086790


In [26]:
data_allMMscaled = df.copy()
data_allMMscaled.iloc[:, 2:] = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df.iloc[:, 2:]), 
                               index=df.iloc[:, 2:].index, 
                               columns=df.iloc[:, 2:].columns)
data_allMMscaled.head()

,HasGarden,GardenCounts_MMscaled,Gender_Female,Age_Over65,Unemployed,Race_Hisp,Race_White,Race_Black,Race_Asian,Race_Other,Income_Low,Income_Mid,Income_High,Edu_Other,Edu_Bachelor,Edu_GradProf,Crime_Felonies_unscaled,Crime_Misdemeanor_unscaled,Crime_Violation_unscaled,Crime_Total_unscaled
NTA,,,,,,,,,,,,,,,,,,,,
Bath Beach,0.0,0.000000,0.439052,0.394840,0.323130,0.175535,0.458779,0.012549,0.508914,0.053256,0.448464,0.675263,0.373584,0.659842,0.433204,0.196127,0.010496,0.012187,0.016769,0.012089
Bay Ridge,0.0,0.000000,0.566993,0.421496,0.343076,0.177737,0.667921,0.020197,0.204717,0.091593,0.412601,0.627940,0.430878,0.494148,0.549841,0.366046,0.011752,0.015232,0.027670,0.015315
Bedford,1.0,0.638889,0.534108,0.188160,0.572512,0.196904,0.367317,0.448119,0.042294,0.074355,0.661452,0.365832,0.254953,0.678467,0.384149,0.208008,0.028667,0.023144,0.038305,0.026313
Bensonhurst East,1.0,0.027778,0.522524,0.423917,0.390147,0.112777,0.488679,0.008212,0.545174,0.067339,0.563712,0.636602,0.261058,0.704291,0.384671,0.165408,0.010267,0.008911,0.017379,0.010138
Bensonhurst West,1.0,0.055556,0.520255,0.380796,0.404529,0.139420,0.448015,0.008740,0.571102,0.054431,0.537315,0.613423,0.298855,0.755354,0.346044,0.115354,0.009682,0.008536,0.015215,0.009528


# Export datasets

In [27]:
data.shape

(191, 28)

In [28]:
data.columns

Index(['HasGarden', 'GardenCounts_MMscaled', 'Gender_Female', 'Age_Over65',
       'Unemployed', 'Race_Hisp', 'Race_White', 'Race_Black', 'Race_Asian',
       'Race_Other', 'Income_Low', 'Income_Mid', 'Income_High',
       'Edu_UnderBach', 'Edu_Bachelor', 'Edu_GradProf',
       'Crime_Felonies_unscaled', 'Crime_Misdemeanor_unscaled',
       'Crime_Violation_unscaled', 'Crime_Total_unscaled',
       'Crime_Felony_scaled', 'Crime_Misdemeanor_scaled',
       'Crime_Violation_scaled', 'Crime_Total_scaled', 'Crime_Felony_MMscaled',
       'Crime_Misdemeanor_MMscaled', 'Crime_Violation_MMscaled',
       'Crime_Total_MMscaled'],
      dtype='object')

In [29]:
data.to_csv('../data/Data_all.csv')

In [30]:
data[['HasGarden', 'GardenCounts_MMscaled', 'Gender_Female', 'Age_Over65',
       'Unemployed', 'Race_Hisp', 'Race_White', 'Race_Black', 'Race_Asian',
       'Race_Other', 'Income_Low', 'Income_Mid', 'Income_High', 'Edu_UnderBach',
       'Edu_Bachelor', 'Edu_GradProf', 'Crime_Felonies_unscaled',
       'Crime_Misdemeanor_unscaled', 'Crime_Violation_unscaled',
       'Crime_Total_unscaled']].to_csv('../data/Data_crimeUnscaled.csv')

In [31]:
data[['HasGarden', 'GardenCounts_MMscaled', 'Gender_Female', 'Age_Over65',
       'Unemployed', 'Race_Hisp', 'Race_White', 'Race_Black', 'Race_Asian',
       'Race_Other', 'Income_Low', 'Income_Mid', 'Income_High', 'Edu_UnderBach',
       'Edu_Bachelor', 'Edu_GradProf','Crime_Felony_scaled',
       'Crime_Misdemeanor_scaled', 'Crime_Violation_scaled',
       'Crime_Total_scaled']].to_csv('../data/Data_crimeStandardScaled.csv')

In [32]:
data[['HasGarden', 'GardenCounts_MMscaled', 'Gender_Female', 'Age_Over65',
       'Unemployed', 'Race_Hisp', 'Race_White', 'Race_Black', 'Race_Asian',
       'Race_Other', 'Income_Low', 'Income_Mid', 'Income_High', 'Edu_UnderBach',
       'Edu_Bachelor', 'Edu_GradProf','Crime_Felony_MMscaled',
       'Crime_Misdemeanor_MMscaled', 'Crime_Violation_MMscaled',
       'Crime_Total_MMscaled']].to_csv('../data/Data_crimeMinMaxScaled.csv')

In [33]:
data_allMMscaled.to_csv('../data/Data_allMinMaxScaled.csv')